In [1]:
# torchtext.legacy를 사용할 수 있는 torchtext 버전 설치
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 5.0 MB/s 
     |████████████████████████████████| 831.4 MB 2.5 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [3]:
#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from torchtext.legacy import data
import torchtext.datasets as datasets

In [15]:
class CNN_Text(nn.Module):
  def __init__(self,embed_num,class_num):
    super(CNN_Text, self).__init__() # nn.Module의 변수 상속
    # V: 사전의 크기
    # D: embed_dim
    # C: 분류하고자 하는 클래스의 개수
    # Co : 각 커널(필터)의 갯수
    V = embed_num
    D = 100
    C = class_num
    Co = 100
    Ks = [3,4,5]

    self.embed = nn.Embedding(V,D)
    self.convs1 = nn.ModuleList([nn.Conv2d(1,Co,(K,100)) for K in Ks])
    self.dropout = nn.Dropout(0.2)
    self.fc1 = nn.Linear(len(Ks)*Co,C)
  
  def forward(self, x):
    x = self.embed(x)
    x = x.unsqueeze(1)
    x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]
    x = [F.max_pool1d(i,i.size(2)).squeeze(2) for i in x]
    x = torch.cat(x,1)
    x = self.dropout(x)
    logit = self.fc1(x)
    return logit


In [5]:
class mydataset(data.Dataset):
  @staticmethod
  def sort_key(ex):
    return len(ex.text)
  def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
    fields = [('text', text_field),('label',label_field)]
    if examples is None:
      path = self.dirname if path is None else path
      examples = []
      for i,line in enumerate(open(path,'r',encoding='utf-8')):
        if i==0:
          continue
        line = line.strip().split('\t')
        txt = line[1].split(' ')

        examples += [data.Example.fromlist([txt,line[2]],fields)]
    super(mydataset, self).__init__(examples,fields,**kwargs)
    

In [8]:
text_field = data.Field(batch_first = True,fix_length=20) # 전처리 관련 field 객체 생성 / fix_length : 하나의 문장 내 max 토큰 수
label_field = data.Field(sequential=False,batch_first = True, unk_token = None) # 전처리 관련 field 객체 생성 / sequential : 시퀸스데이터 여부

train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_train_tok.txt')
test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_test_tok.txt')

text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

train_iter, test_iter = data.Iterator.splits(
    (train_data,test_data),batch_sizes=(100,1)
)

print(len(text_field.vocab))


21893


In [16]:
cnn = CNN_Text(len(text_field.vocab),2)

optimizer = torch.optim.Adam(cnn.parameters())
cnn.train() #학습 모드

CNN_Text(
  (embed): Embedding(21893, 100)
  (convs1): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=300, out_features=2, bias=True)
)

In [17]:
for epoch in range(20):
  totalloss = 0

  for batch in train_iter:
    optimizer.zero_grad() # 그래디언트 초기화

    txt = batch.text
    label = batch.label

    pred = cnn(txt)

    loss = F.cross_entropy(pred,label)
    totalloss += loss.data

    loss.backward()

    optimizer.step()

  print(epoch,'eopch')
  print('loss : {:.3f}'.format(totalloss.numpy()))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


0 eopch
loss : 65.933
1 eopch
loss : 50.602
2 eopch
loss : 36.296
3 eopch
loss : 24.258
4 eopch
loss : 15.471
5 eopch
loss : 11.195
6 eopch
loss : 7.853
7 eopch
loss : 5.805
8 eopch
loss : 4.459
9 eopch
loss : 3.695
10 eopch
loss : 3.448
11 eopch
loss : 2.799
12 eopch
loss : 2.296
13 eopch
loss : 2.046
14 eopch
loss : 2.079
15 eopch
loss : 1.708
16 eopch
loss : 1.634
17 eopch
loss : 1.301
18 eopch
loss : 1.364
19 eopch
loss : 1.643


In [18]:
%%time
from sklearn.metrics import classification_report

cnn.eval() # 검증 모드
correct = 0
incorrect = 0
y_test = []
prediction = []

for batch in test_iter:
  txt = batch.text
  label = batch.label
  y_test.append(label.data[0])

  pred = cnn(txt)
  _,ans = torch.max(pred,dim=1)
  prediction.append(ans.data[0])

  if ans.data[0] == label.data[0]:
    correct += 1
  else :
    incorrect += 1

print('correct : ',correct) 
print('incorrect : ',incorrect) 
print(classification_report(torch.tensor(y_test),
                            torch.tensor(prediction),
                            digits=4,
                            target_names=['negative','positive']))

correct :  79
incorrect :  21
              precision    recall  f1-score   support

    negative     0.8222    0.7400    0.7789        50
    positive     0.7636    0.8400    0.8000        50

    accuracy                         0.7900       100
   macro avg     0.7929    0.7900    0.7895       100
weighted avg     0.7929    0.7900    0.7895       100

CPU times: user 756 ms, sys: 94.2 ms, total: 850 ms
Wall time: 1.55 s
